# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Get CSV filepaths

In [2]:
# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Loop through all files and collect links for each file
for root, dirs, files in os.walk(filepath):
    
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# Initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 

# For every filepath in the file path list 
for f in file_path_list:

    # Read csv file 
    with open(f, 'r', encoding = 'utf8', newline = '') as csvfile: 
        # Create a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # Extract each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line)

# Select the correct appropriate columns and get rid of rows empty rows
# Write into a new file called "event_datdafile_new.csv"
csv.register_dialect('myDialect', quoting = csv.QUOTE_ALL, skipinitialspace = True)
with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline = '') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                     'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

# Part II. Cassandra Data Modeling and Insert Data

#### Creating a Cluster

In [4]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace and Set Key Sapce

In [5]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")
    session.set_keyspace('sparkify')

except Exception as e:
    print(e)

## Queries

#### Before we start data modeling with Apache Cassandra, we must first understand how a user will interact with the database. To start, the analytics team at Sparkify wants to understand three basic questions:

1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4.
2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182.
3. Give me every user name (first and last) in my music app history who listened to the song "All Hands Against His Own".

Therefore, we will create one table for each of the above queries.

#### Query 1
#### "Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4."

Because `sessionId` and `itemInSession` both appear in the `WHERE` clause, it is natural to use them in the primary key.

In [6]:
# Data file
file = 'event_datafile_new.csv'

In [7]:
# Create table
query = "CREATE TABLE IF NOT EXISTS query_1 "
query = query + "(artist text, song_title text, song_length float, session_id int, item_in_session int, "
query = query + "PRIMARY KEY (session_id, item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [8]:
# Insert data
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO query_1 (artist, song_title, song_length, session_id, item_in_session) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

In [9]:
# Query data to check for correctness
try:
    rows = session.execute("""
        SELECT 
            artist,
            song_title,
            song_length
        FROM query_1
        WHERE session_id = 338 AND
              item_in_session = 4
    """)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.song_length, sep = '|')

Faithless|Music Matters (Mark Knight Dub)|495.30731201171875


#### Query 2
#### "Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182."

Because `userId` and `sessionId` both appear in the `WHERE` clause, it is natural to use them in the primary key. We also add `itemInSession` as a clustering column because the query wants the `song` column to appear sorted. Do this can increase performance because the clustering column presorts the data as they are inserted.

In [10]:
# Create Table
query = "CREATE TABLE IF NOT EXISTS query_2 "
query = query + "(artist text, song_title text, first_name text, last_name text, user_id int, session_id int, item_in_session int, "
query = query + "PRIMARY KEY (user_id, session_id, item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [11]:
# Insert Data
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO query_2 (artist, song_title, first_name, last_name, user_id, session_id, item_in_session) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], line[1], line[4], int(line[10]), int(line[8]), int(line[3])))

In [12]:
# Query data to check for correctness
try:
    rows = session.execute("""
        SELECT 
            artist,
            song_title,
            first_name,
            last_name,
            item_in_session --Add this column just to show that it is sorted
        FROM query_2
        WHERE user_id = 10 AND
              session_id = 182
    """)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.first_name, row.last_name, row.item_in_session, sep = '|')

Down To The Bone|Keep On Keepin' On|Sylvie|Cruz|0
Three Drives|Greece 2000|Sylvie|Cruz|1
Sebastien Tellier|Kilometer|Sylvie|Cruz|2
Lonnie Gordon|Catch You Baby (Steve Pitron & Max Sanna Radio Edit)|Sylvie|Cruz|3


#### Query 3
#### "Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'"

For this query, we can set the composite primary key to `song` and `userId`. Since we are only interested in whether or not a user has listened to a particular song, we do not have to worry about duplication when a a user listens to the same song more than once. A record with a duplicate primary key will automatically overwrite the original record.

In [13]:
# Create data
query = "CREATE TABLE IF NOT EXISTS query_3 "
query = query + "(first_name text, last_name text, user_id int, song_title text, "
query = query + "PRIMARY KEY (song_title, user_id))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [14]:
# Insert data
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO query_3 (first_name, last_name, user_id, song_title) "
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[1], line[4], int(line[10]), line[9]))

In [16]:
# Query data to check for correctness
try:
    rows = session.execute("""
        SELECT 
            first_name, 
            last_name
        FROM query_3
        WHERE song_title = 'All Hands Against His Own'
    """)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name, sep='|')

Tegan|Levine
Sara|Johnson


## Drop the tables before closing out the sessions

In [17]:
tables = ["query_1", "query_2", "query_3"]

for table in tables:
    query = 'DROP TABLE ' + table
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)